1. 데이터 RDD 생성 map()
2. collect()
3. blue mnm만 꺼내기
4. 전체 합계 구하기
5. 도시별 합계 구하기

In [2]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("mnm-data")
spark = SparkContext(conf=conf).getOrCreate()

24/12/03 15:33:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50748)
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/envs/spark_start/lib/python3.8/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/home/ubuntu/anaconda3/envs/spark_start/lib/python3.8/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/home/ubuntu/anaconda3/envs/spark_start/lib/python3.8/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  Fi

In [3]:
import os
directory = os.path.join(os.getcwd(),"data")
filename = "mnm_dataset.csv"
filepath = os.path.join(directory, filename)
lines = spark.textFile("file:///"+filepath.replace("\\","/"))
lines

file:////home/lab12/src/data/mnm_dataset.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [4]:
lines.take(5)

['State,Color,Count', 'TX,Red,20', 'NV,Blue,66', 'CO,Blue,79', 'OR,Blue,71']

In [5]:
#데이터 변환

In [14]:
data_rdd = lines.map(lambda line: line.split(","))
data_rdd

PythonRDD[17] at RDD at PythonRDD.scala:53

In [7]:
# 튜플 변환
structured_rdd = data_rdd.map(lambda fields: (fields[0], fields[1], int(fields[2])))

In [16]:
sample_data = structured_rdd.take(10)
sample_data

[['State', 'Color', 'Count'],
 ['TX', 'Red', '20'],
 ['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WA', 'Yellow', '93'],
 ['WY', 'Blue', '16'],
 ['CA', 'Yellow', '53'],
 ['WA', 'Green', '60'],
 ['OR', 'Green', '71']]

In [15]:
# 'Blue' M&M만 필터링
blue_mnm_rdd = structured_rdd.filter(lambda row: row[1] == "Blue")

# 결과 확인 (일부 데이터만 출력)
blue_mnm_rdd.take(10)


[['NV', 'Blue', '66'],
 ['CO', 'Blue', '79'],
 ['OR', 'Blue', '71'],
 ['WY', 'Blue', '16'],
 ['AZ', 'Blue', '75'],
 ['CO', 'Blue', '52'],
 ['CO', 'Blue', '95'],
 ['CO', 'Blue', '98'],
 ['CA', 'Blue', '13'],
 ['NV', 'Blue', '50']]

In [13]:
# 도시(State)별 Count 합계 계산
state_totals = structured_rdd.map(lambda row: (row[0], row[2])).reduceByKey(lambda x, y: x + y)

state_totals


PythonRDD[16] at RDD at PythonRDD.scala:53

In [ ]:
spark.stop()